In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname in os.listdir('/kaggle/input'):
    for filename in os.listdir(os.path.join("/kaggle/input", dirname)):
        print(os.path.join("kaggle", "input", dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

kaggle/input/distilbertdistilbert-base-uncased/transformers
kaggle/input/distilbertdistilbert-base-uncased-finetuned/transformers
kaggle/input/jigsaw/subreddits.csv
kaggle/input/jigsaw/features.csv
kaggle/input/jigsaw-agile-community-rules/sample_submission.csv
kaggle/input/jigsaw-agile-community-rules/train.csv
kaggle/input/jigsaw-agile-community-rules/test.csv
kaggle/input/facebookai-roberta-large-mnli/transformers
kaggle/input/pm-87757141-at-07-27-2025-17-07-03/__script__.py
kaggle/input/pm-87757141-at-07-27-2025-17-07-03/charset_normalizer-3.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
kaggle/input/pm-87757141-at-07-27-2025-17-07-03/aiohttp-3.12.14-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
kaggle/input/pm-87757141-at-07-27-2025-17-07-03/certifi-2025.7.14-py3-none-any.whl
kaggle/input/pm-87757141-at-07-27-2025-17-07-03/filelock-3.18.0-py3-none-any.whl
kaggle/input/pm-87757141-at-07-27-2025-17-07-03/evaluate-0.4.5-py3-none-any.whl
kaggle/input/pm-87

# imports

In [2]:
import wandb
wandb.init(mode="disabled")

In [3]:
import heapq
import json
import logging
import os
import re
import sys
from datetime import datetime
from functools import partial
from pathlib import Path
from typing import Any, Callable, Dict, List, Mapping, Optional, Tuple, Union
import warnings

import markdown2
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from bs4 import BeautifulSoup
from datasets import ClassLabel, Dataset, DatasetDict, concatenate_datasets
from scipy.special import softmax
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from torchinfo import summary
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainerCallback,
    TrainingArguments,
    pipeline,
)
from unidecode import unidecode

2025-07-27 17:09:05.547939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753636145.830442      26 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753636145.920379      26 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
os.environ["WANDB_DISABLED"] = "true"
warnings.simplefilter("ignore")

# load dataset

In [5]:
INPUT_PATH = os.path.join("/", "kaggle", "input")
train = pd.read_csv(os.path.join(INPUT_PATH, "jigsaw-agile-community-rules", "train.csv"))
test = pd.read_csv(os.path.join(INPUT_PATH, "jigsaw-agile-community-rules", "test.csv"))
submission = pd.read_csv(os.path.join(INPUT_PATH, "jigsaw-agile-community-rules", "sample_submission.csv"))
features = pd.read_csv(os.path.join(INPUT_PATH, "jigsaw", "features.csv"))["features"].tolist()
subreddits = pd.read_csv(os.path.join(INPUT_PATH, "jigsaw", "subreddits.csv"))["subreddit"].tolist()

# util funcs

In [6]:
def clean_mem():
    import sys
    import gc
    import traceback
    import torch
    import psutil
    import os
    import time

    process = psutil.Process(os.getpid())

    # Measure RAM before cleanup
    ram_before = process.memory_info().rss / (1024**2)  # in MB

    # Measure GPU before cleanup
    if torch.cuda.is_available():
        gpu_alloc_before = torch.cuda.memory_allocated() / (1024**2)  # in MB
        gpu_reserved_before = torch.cuda.memory_reserved() / (1024**2)  # in MB
    else:
        gpu_alloc_before = gpu_reserved_before = 0

    # clean all traceback
    if hasattr(sys, "last_traceback"):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, "last_traceback")
    if hasattr(sys, "last_type"):
        delattr(sys, "last_type")
    if hasattr(sys, "last_value"):
        delattr(sys, "last_value")

    # clean all ipython history
    if "get_ipython" in globals():
        ip = get_ipython()
        user_ns = ip.user_ns
        ip.displayhook.flush()
        pc = ip.displayhook.prompt_count + 1
        for n in range(1, pc):
            user_ns.pop("_i" + repr(n), None)
        user_ns.update(dict(_i="", _ii="", _iii=""))
        hm = ip.history_manager
        hm.input_hist_parsed[:] = [""] * pc
        hm.input_hist_raw[:] = [""] * pc
        hm._i = hm._ii = hm._iii = hm._i00 = ""

    # do a garbage collection and flush cuda cache
    gc.collect()
    torch.cuda.empty_cache()

    # Give system a small moment to settle (helps RAM measurement be more accurate)
    time.sleep(0.1)

    # Measure RAM after cleanup
    ram_after = process.memory_info().rss / (1024**2)  # in MB

    # Measure GPU after cleanup
    if torch.cuda.is_available():
        gpu_alloc_after = torch.cuda.memory_allocated() / (1024**2)  # in MB
        gpu_reserved_after = torch.cuda.memory_reserved() / (1024**2)  # in MB
    else:
        gpu_alloc_after = gpu_reserved_after = 0

    # Report freed memory
    print(
        f"RAM freed: {ram_before - ram_after:.2f} MB ({ram_before:.2f} -> {ram_after:.2f})"
    )
    if torch.cuda.is_available():
        print(
            f"GPU allocated freed: {gpu_alloc_before - gpu_alloc_after:.2f} MB ({gpu_alloc_before:.2f} -> {gpu_alloc_after:.2f})"
        )
        print(
            f"GPU reserved freed: {gpu_reserved_before - gpu_reserved_after:.2f} MB ({gpu_reserved_before:.2f} -> {gpu_reserved_after:.2f})"
        )
    else:
        print("No GPU detected.")


clean_mem()

RAM freed: 0.00 MB (1413.05 -> 1413.05)
GPU allocated freed: 0.00 MB (0.00 -> 0.00)
GPU reserved freed: 0.00 MB (0.00 -> 0.00)


In [7]:
def create_logger(
    name: str = "reddit_moderation",
    log_level: str = "INFO",
    log_file: Optional[Union[str, Path]] = None,
    log_dir: Optional[Union[str, Path]] = "logs",
    console_output: bool = True,
    file_output: bool = True,
    format_string: Optional[str] = None,
    max_bytes: int = 10_000_000,  # 10MB
    backup_count: int = 5,
    include_timestamp_in_filename: bool = True,
) -> logging.Logger:
    """
    Create a fully featured logger for the Reddit comment moderation system.

    This logger is designed to handle all aspects of the multi-stage classification
    pipeline including zero-shot classification, fine-tuning, and evaluation.

    Parameters
    ----------
    name : str, optional
        Logger name, by default "reddit_moderation"
    log_level : str, optional
        Logging level ("DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"),
        by default "INFO"
    log_file : str or Path, optional
        Specific log file path. If None, auto-generates based on name and timestamp
    log_dir : str or Path, optional
        Directory for log files, by default "logs"
    console_output : bool, optional
        Whether to output logs to console, by default True
    file_output : bool, optional
        Whether to output logs to file, by default True
    format_string : str, optional
        Custom log format string, by default None (uses comprehensive format)
    max_bytes : int, optional
        Maximum log file size before rotation, by default 10MB
    backup_count : int, optional
        Number of backup log files to keep, by default 5
    include_timestamp_in_filename : bool, optional
        Whether to include timestamp in log filename, by default True

    Returns
    -------
    logging.Logger
        Configured logger instance ready for use

    Examples
    --------
    >>> # Basic usage
    >>> logger = create_logger()
    >>> logger.info("Starting Reddit comment classification pipeline")

    >>> # Advanced usage for training
    >>> training_logger = create_logger(
    ...     name="distilbert_training",
    ...     log_level="DEBUG",
    ...     log_file="training_session.log"
    ... )
    >>> training_logger.debug("Training batch processed")

    >>> # For evaluation only
    >>> eval_logger = create_logger(
    ...     name="model_evaluation",
    ...     console_output=False,
    ...     log_file="evaluation_results.log"
    ... )
    """

    # Create logger
    logger = logging.getLogger(name)
    logger.setLevel(getattr(logging, log_level.upper()))

    # Clear existing handlers to avoid duplication
    logger.handlers.clear()

    # Default comprehensive format for ML workflows
    if format_string is None:
        format_string = (
            "%(asctime)s | %(name)s | %(levelname)s | %(message)s"
        )

    formatter = logging.Formatter(format_string, datefmt="%Y-%m-%d %H:%M:%S")

    # Console handler
    if console_output:
        console_handler = logging.StreamHandler(sys.stdout)
        console_handler.setLevel(getattr(logging, log_level.upper()))
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)

    # File handler with rotation
    if file_output:
        # Create log directory
        if log_dir:
            log_dir = Path(log_dir)
            log_dir.mkdir(exist_ok=True)

        # Generate log filename
        if log_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            if include_timestamp_in_filename:
                log_filename = f"{name}_{timestamp}.log"
            else:
                log_filename = f"{name}.log"
            log_file = log_dir / log_filename if log_dir else Path(log_filename)
        else:
            log_file = Path(log_file)
            if log_dir and not log_file.is_absolute():
                log_file = Path(log_dir) / log_file

        # Create rotating file handler
        from logging.handlers import RotatingFileHandler

        file_handler = RotatingFileHandler(
            log_file, maxBytes=max_bytes, backupCount=backup_count, encoding="utf-8"
        )
        file_handler.setLevel(getattr(logging, log_level.upper()))
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)

    # Add some useful methods to the logger
    def log_dataset_info(dataset, dataset_name="Dataset"):
        """Log dataset information"""
        logger.info(f"{dataset_name} Info:")
        logger.info(f"  - Size: {len(dataset):,} samples")
        logger.info(f"  - Columns: {dataset.column_names}")
        if "labels" in dataset.column_names:
            import numpy as np

            labels = np.array(dataset["labels"])
            unique, counts = np.unique(labels, return_counts=True)
            logger.info(f"  - Label distribution: {dict(zip(unique, counts))}")

    def log_model_info(model, model_name="Model"):
        """Log model information"""
        logger.info(f"{model_name} Info:")
        if hasattr(model, "config"):
            logger.info(f"  - Model type: {model.config.model_type}")
            logger.info(f"  - Hidden size: {model.config.hidden_size}")
            if hasattr(model.config, "num_labels"):
                logger.info(f"  - Number of labels: {model.config.num_labels}")

        # Count parameters
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        logger.info(f"  - Total parameters: {total_params:,}")
        logger.info(f"  - Trainable parameters: {trainable_params:,}")

    def log_training_args(training_args):
        """Log training arguments"""
        logger.info("Training Configuration:")
        logger.info(f"  - Learning rate: {training_args.learning_rate}")
        logger.info(f"  - Batch size: {training_args.per_device_train_batch_size}")
        logger.info(
            f"  - Gradient accumulation: {training_args.gradient_accumulation_steps}"
        )
        logger.info(f"  - Epochs: {training_args.num_train_epochs}")
        logger.info(f"  - Weight decay: {training_args.weight_decay}")
        logger.info(f"  - LR scheduler: {training_args.lr_scheduler_type}")
        logger.info(f"  - Warmup ratio: {training_args.warmup_ratio}")

    def log_metrics(metrics, stage=""):
        """Log evaluation metrics"""
        stage_prefix = f"{stage} " if stage else ""
        logger.info(f"{stage_prefix}Metrics:")
        for metric, value in metrics.items():
            if isinstance(value, float):
                logger.info(f"  - {metric}: {value:.4f}")
            else:
                logger.info(f"  - {metric}: {value}")

    # Attach utility methods to logger
    logger.log_dataset_info = log_dataset_info
    logger.log_model_info = log_model_info
    logger.log_training_args = log_training_args
    logger.log_metrics = log_metrics

    # Log logger creation
    logger.info(f"Logger '{name}' created successfully")
    logger.info(f"Log level: {log_level}")
    if file_output:
        logger.info(f"Log file: {log_file}")

    return logger


# Convenience function for quick setup
def setup_project_logging(debug_mode: bool = False) -> logging.Logger:
    """
    Quick setup for the Reddit moderation project logging.

    Parameters
    ----------
    debug_mode : bool
        If True, sets log level to DEBUG and enables verbose logging

    Returns
    -------
    logging.Logger
        Configured project logger
    """
    log_level = "DEBUG" if debug_mode else "INFO"

    return create_logger(
        name="reddit_moderation_pipeline",
        log_level=log_level,
        log_dir="project_logs",
        include_timestamp_in_filename=True,
    )

# models

In [8]:
# Basic setup for your notebook
logger = create_logger()
logger.info("Starting Context-Aware Reddit Comment Moderation System")

# Training-specific logger
training_logger = create_logger(
    name="distilbert_finetuning",
    log_level="DEBUG",
    log_file="training_session.log"
)

2025-07-27 17:09:27 | reddit_moderation | INFO | Logger 'reddit_moderation' created successfully
2025-07-27 17:09:27 | reddit_moderation | INFO | Log level: INFO
2025-07-27 17:09:27 | reddit_moderation | INFO | Log file: logs/reddit_moderation_20250727_170927.log
2025-07-27 17:09:27 | reddit_moderation | INFO | Starting Context-Aware Reddit Comment Moderation System
2025-07-27 17:09:27 | distilbert_finetuning | INFO | Logger 'distilbert_finetuning' created successfully
2025-07-27 17:09:27 | distilbert_finetuning | INFO | Log level: DEBUG
2025-07-27 17:09:27 | distilbert_finetuning | INFO | Log file: logs/training_session.log


In [9]:
MODEL_PATH = {
    # "classifier": "/kaggle/input/distilbertdistilbert-base-uncased/transformers/default/1",
    "classifier": "/kaggle/input/facebookai-roberta-large-mnli/transformers/default/1",
    "nli": "/kaggle/input/moritzlaurerdeberta-v3-base-mnli-fever-anli/transformers/default/1",
}

In [10]:
logger.info(f"{MODEL_PATH = }")

2025-07-27 17:09:27 | reddit_moderation | INFO | MODEL_PATH = {'classifier': '/kaggle/input/facebookai-roberta-large-mnli/transformers/default/1', 'nli': '/kaggle/input/moritzlaurerdeberta-v3-base-mnli-fever-anli/transformers/default/1'}


# training dataset prep

In [11]:
def sanitize_comment(comment):
    # Convert markdown to HTML, then extract the text (HTML tags removed)
    html = markdown2.markdown(comment)
    text = BeautifulSoup(html, features="html.parser").get_text()

    # Convert markdown links [text](url) to just "text"
    # Must be done on original comment, but here we do it on extracted text to be safe
    # To be sure, you can do it before markdown conversion, but here kept as is for simplicity

    # The markdown2 conversion often converts markdown links into HTML anchors,
    # so links should already have URL removed by BeautifulSoup.get_text().
    # However, just in case, let's remove leftover markdown links from original comment first:
    text = re.sub(r"\[([^\]]+)\]\(([^)]+)\)", r"\1", comment)
    # Then re-run markdown2 and extract text again to clean up
    html = markdown2.markdown(text)
    text = BeautifulSoup(html, features="html.parser").get_text()

    # Replace URLs with the url itself as plain text
    # Extract URLs and replace markdown-style inline URLs [text](url) with url only is already handled,
    # but explicit URLs just in text should be replaced with the URL string, not removed.
    # For example: "visit http://example.com for more" should keep "http://example.com" as is.
    # So to convert URL markdown to plain URLs requires us to find URLs and keep them as text.

    # Here, let's find URLs and replace any markdown link forms to plain URLs if any missed:
    # But since markdown2 and BeautifulSoup stripped them to plain text, raw URLs remain intact.

    # So no need to remove URLs, but ensure any URLs embedded in text like "https://..." remain
    # We can optionally extract and re-insert URLs if you want, but seems not required.

    # Just to be sure, let's convert all URL-like substrings to themselves surrounded by spaces (to separate)
    # This helps if URLs are concatenated with other text.
    url_pattern = re.compile(r"((?:http|https)://[^\s]+|www\.[^\s]+)", re.IGNORECASE)
    text = url_pattern.sub(lambda m: m.group(0), text)

    # Convert non-unicode characters to unicode (ASCII compatible)
    text = unidecode(text)

    # Normalize whitespace
    text = " ".join(text.split()).lower()

    return text




In [12]:
def create_master_dataset(
    train: pd.DataFrame, 
    test: pd.DataFrame, 
    logger: Optional[logging.Logger] = None
) -> pd.DataFrame:
    if logger:
        logger.info("Starting master dataset creation")
        logger.info(f"Input - Train: {len(train)} rows, Test: {len(test)} rows")
    
    # 1. From train: use body
    if logger:
        logger.debug("Extracting main training data from 'body' column")
    train_main = train[["body", "rule", "subreddit", "rule_violation"]].copy()
    train_main = train_main.rename(
        columns={"body": "comment", "rule_violation": "violation"}
    )
    if logger:
        logger.debug(f"Main training data: {len(train_main)} records")

    # 2. From train AND test: from positive/negative examples

    # Helper to melt examples from a single dataframe
    def extract_examples(
        df, prefix_pos="positive_example_", prefix_neg="negative_example_"
    ):
        records = []

        # For positive examples
        for i in [1, 2]:
            col = f"{prefix_pos}{i}"
            # Ensure column exists and drop NA
            if col in df.columns:
                subdf = df[["rule", "subreddit", col]].dropna(subset=[col])
                if logger:
                    logger.debug(f"Extracting {len(subdf)} positive examples from {col}")
                for _, row in subdf.iterrows():
                    records.append(
                        {
                            "comment": row[col],
                            "rule": row["rule"],
                            "subreddit": row["subreddit"],
                            "violation": 1,
                        }
                    )

        # For negative examples
        for i in [1, 2]:
            col = f"{prefix_neg}{i}"
            if col in df.columns:
                subdf = df[["rule", "subreddit", col]].dropna(subset=[col])
                if logger:
                    logger.debug(f"Extracting {len(subdf)} negative examples from {col}")
                for _, row in subdf.iterrows():
                    records.append(
                        {
                            "comment": row[col],
                            "rule": row["rule"],
                            "subreddit": row["subreddit"],
                            "violation": 0,
                        }
                    )

        return pd.DataFrame(records)

    if logger:
        logger.debug("Extracting examples from train dataset")
    train_examples = extract_examples(train)
    if logger:
        logger.debug(f"Train examples extracted: {len(train_examples)} records")
    
    if logger:
        logger.debug("Extracting examples from test dataset")
    test_examples = extract_examples(test)
    if logger:
        logger.debug(f"Test examples extracted: {len(test_examples)} records")

    # Concatenate all parts
    if logger:
        logger.info("Concatenating all dataset parts")
    master_df = pd.concat(
        [train_main, train_examples, test_examples], ignore_index=True
    )

    # Optional: drop rows with empty or null comment if any sneaked in
    initial_size = len(master_df)
    master_df = master_df.dropna(subset=["comment"])
    if logger and len(master_df) < initial_size:
        logger.warning(f"Dropped {initial_size - len(master_df)} rows with null comments")

    # Reset index for cleanliness
    master_df = master_df.reset_index(drop=True)

    if logger:
        logger.info(f"Master dataset created successfully: {len(master_df)} total records")
        logger.info(f"Violation distribution: {master_df['violation'].value_counts().to_dict()}")

    return master_df


In [13]:
for c in [
    "body",
    "rule",
    "subreddit",
    "positive_example_1",
    "positive_example_2",
    "negative_example_1",
    "negative_example_2",
]:
    logger.info(f"Cleaning {c = }")
    train[c] = train[c].apply(sanitize_comment)
    test[c] = test[c].apply(sanitize_comment)

2025-07-27 17:09:27 | reddit_moderation | INFO | Cleaning c = 'body'
2025-07-27 17:09:28 | reddit_moderation | INFO | Cleaning c = 'rule'
2025-07-27 17:09:29 | reddit_moderation | INFO | Cleaning c = 'subreddit'
2025-07-27 17:09:29 | reddit_moderation | INFO | Cleaning c = 'positive_example_1'
2025-07-27 17:09:30 | reddit_moderation | INFO | Cleaning c = 'positive_example_2'
2025-07-27 17:09:31 | reddit_moderation | INFO | Cleaning c = 'negative_example_1'
2025-07-27 17:09:32 | reddit_moderation | INFO | Cleaning c = 'negative_example_2'


In [14]:
master_dataset = create_master_dataset(train, test, logger)

2025-07-27 17:09:33 | reddit_moderation | INFO | Starting master dataset creation
2025-07-27 17:09:33 | reddit_moderation | INFO | Input - Train: 2029 rows, Test: 10 rows
2025-07-27 17:09:34 | reddit_moderation | INFO | Concatenating all dataset parts
2025-07-27 17:09:34 | reddit_moderation | INFO | Master dataset created successfully: 10185 total records
2025-07-27 17:09:34 | reddit_moderation | INFO | Violation distribution: {1: 5109, 0: 5076}


In [15]:
master_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10185 entries, 0 to 10184
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   comment    10185 non-null  object
 1   rule       10185 non-null  object
 2   subreddit  10185 non-null  object
 3   violation  10185 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 318.4+ KB


In [16]:
dataset = Dataset.from_pandas(master_dataset)

# extract features

In [17]:
MODEL_PATH["nli"]

'/kaggle/input/moritzlaurerdeberta-v3-base-mnli-fever-anli/transformers/default/1'

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# rule_classifier = pipeline(
#     "zero-shot-classification", model=MODEL_PATH["nli"], device=device
# )
subreddit_classifier = pipeline(
    "zero-shot-classification", model=MODEL_PATH["nli"], device=device
)
rule_vectorizer = TfidfVectorizer().fit(features)
rule_vecs = rule_vectorizer.transform(features)

# subreddit_vectorizer = TfidfVectorizer().fit(subreddits)
# subreddit_vecs = subreddit_vectorizer.transform(subreddits)

Device set to use cuda


In [19]:
def predict_labels(
    batch,  # ← comes from a HF Dataset
    vectorizer,
    label_vecs,
    column: str,
    labels: list,
    threshold: float = 0.5,
    heap_nlargest: int = 2,
    logger: Optional[logging.Logger] = None,
):
    if logger:
        logger.info(f"Starting prediction for column '{column}'")
        
    # encode
    texts = batch[column] if isinstance(batch[column], list) else [batch[column]]
    if logger:
        logger.info(f"Processing {len(texts)} texts from batch")
        
    rule_vecs = vectorizer.transform(texts)
    if logger:
        logger.debug(f"Vectorized texts: shape {rule_vecs.shape}")
        
    sims = (rule_vecs @ label_vecs.T).toarray()  # cosine similarity (same as sklearn's)
    if logger:
        logger.debug(f"Computed similarity matrix: shape {sims.shape}")

    out = []
    for i, sim_row in enumerate(sims):
        pairs = list(zip(labels, sim_row))
        top2 = heapq.nlargest(heap_nlargest, pairs, key=lambda x: x[1])
        above = [(l, s) for l, s in top2 if s >= threshold]
        chosen = above if above else [max(pairs, key=lambda x: x[1])]
        
        if logger:
            best_scores = [f"{l}:{s:.3f}" for l, s in top2[:3]]  # Top 3 for debug
            logger.info(f"Sample {i}: top scores {best_scores}, "
                        f"above threshold: {len(above)}, chosen: {len(chosen)}")
        
        out.append(", ".join(f"{l}" for l, s in chosen))

    if logger:
        logger.info(f"Prediction completed for {len(out)} samples on column '{column}'")
        logger.info(f"Sample predictions: {out[:3] if len(out) >= 3 else out}")

    return {f"predicted_{column}_feature": out}

def predict_labels_nli(
    batch,
    classifier,
    column: str,
    labels: list,
    threshold: float = 0.8,
    lower_threshold: float = 0.5,  # NEW PARAM ─ default 0.5
    logger: Optional[logging.Logger] = None,
):
    if logger:
        logger.info(f"Starting NLI prediction for column '{column}'")
        
    texts = batch[column] if isinstance(batch[column], list) else [batch[column]]
    if logger:
        logger.info(f"Processing {len(texts)} texts with NLI classifier")
        
    outputs = classifier(texts, labels, multi_label=True)
    if isinstance(outputs, dict):  # single-example edge case
        outputs = [outputs]
        if logger:
            logger.debug("Converted single output dict to list format")

    if logger:
        logger.debug(f"NLI classifier returned {len(outputs)} predictions")

    out = []
    fallback_count = 0
    for i, o in enumerate(outputs):
        pairs = list(zip(o["labels"], o["scores"]))  # (label, score)
        top2 = heapq.nlargest(2, pairs, key=lambda x: x[1])
        above = [(l, s) for l, s in top2 if s >= threshold]
        
        if above:
            chosen = above  # 1 or 2 labels
            if logger:
                logger.debug(f"Sample {i}: {len(above)} labels above threshold {threshold}")
        else:
            top1 = max(pairs, key=lambda x: x[1])  # best label overall
            if top1[1] < lower_threshold:
                chosen = [("general or other", 1.0)]
                fallback_count += 1
                if logger:
                    logger.info(f"Sample {i}: best score {top1[1]:.3f} < {lower_threshold}, using fallback")
            else:
                chosen = [top1]
                if logger:
                    logger.info(f"Sample {i}: using best label {top1[0]} with score {top1[1]:.3f}")
        
        out.append(", ".join(f"{l}" for l, s in sorted(chosen, key=lambda x: -x[1])))

    if logger:
        logger.info(f"NLI prediction completed for {len(out)} samples on column '{column}'")
        if fallback_count > 0:
            logger.info(f"Used fallback 'general or other' for {fallback_count} samples")
        logger.info(f"Sample predictions: {out[:3] if len(out) >= 3 else out}")

    return {f"predicted_{column}_feature": out}

In [20]:
def _calculate_column_lookup(dataset, column, map_fn, **map_kwargs):
    unique_vals = list(set(dataset[column]))  # unique strings
    mini_ds = Dataset.from_dict({column: unique_vals})  # tiny dataset
    scored_ds = mini_ds.map(  # classify once each
        partial(map_fn, column=column, **map_kwargs), batched=True, batch_size=128
    )
    return dict(zip(unique_vals, scored_ds[f"predicted_{column}_feature"]))

In [21]:
def _build_lookups(df, logger=None):
    rule_lookup = _calculate_column_lookup(
        df,  # or val_hf_small
        column="rule",
        map_fn=predict_labels,
        vectorizer=rule_vectorizer,
        label_vecs=rule_vecs,
        labels=features,
        threshold=0.3,
        heap_nlargest=4,
        logger=logger
    )

    subreddit_lookup = _calculate_column_lookup(
        df,
        column="subreddit",
        map_fn=predict_labels_nli,
        classifier=subreddit_classifier,
        labels=subreddits,
        threshold=0.8,
        logger=logger
    )
    return rule_lookup, subreddit_lookup



In [22]:
def _map_lookup(example, rule_lookup, subreddit_lookup):
    example["predicted_rule_feature"] = rule_lookup[example["rule"]]
    example["predicted_subreddit_feature"] = subreddit_lookup[example["subreddit"]]
    return example

In [23]:
def add_classification_preds_rule_subreddit(
    df, rule_lookup=None, subreddit_lookup=None, logger=None
):
    if rule_lookup is None and subreddit_lookup is None:
        rule_lookup, subreddit_lookup = _build_lookups(df, logger=logger)
    result = df.map(
        partial(
            _map_lookup, rule_lookup=rule_lookup, subreddit_lookup=subreddit_lookup
        ),
        desc="Attach predictions",
    )
    return result, rule_lookup, subreddit_lookup

In [24]:
dataset, rule_lookup, subreddit_lookup = add_classification_preds_rule_subreddit(
    dataset, logger=logger
)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

2025-07-27 17:09:38 | reddit_moderation | INFO | Starting prediction for column 'rule'
2025-07-27 17:09:38 | reddit_moderation | INFO | Processing 2 texts from batch
2025-07-27 17:09:38 | reddit_moderation | INFO | Sample 0: top scores ['ban advertising:0.765', 'ban spam:0.383', 'ban illegal content:0.137'], above threshold: 2, chosen: 2
2025-07-27 17:09:38 | reddit_moderation | INFO | Sample 1: top scores ['enforce respectful conduct:0.000', 'enforce on topic content:0.000', 'ban low effort content:0.000'], above threshold: 0, chosen: 1
2025-07-27 17:09:38 | reddit_moderation | INFO | Prediction completed for 2 samples on column 'rule'
2025-07-27 17:09:38 | reddit_moderation | INFO | Sample predictions: ['ban advertising, ban spam', 'enforce respectful conduct']


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

2025-07-27 17:09:42 | reddit_moderation | INFO | Starting NLI prediction for column 'subreddit'
2025-07-27 17:09:42 | reddit_moderation | INFO | Processing 100 texts with NLI classifier


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


2025-07-27 17:10:37 | reddit_moderation | INFO | Sample 0: using best label relationship with score 0.791
2025-07-27 17:10:37 | reddit_moderation | INFO | Sample 7: using best label niche with score 0.672
2025-07-27 17:10:37 | reddit_moderation | INFO | Sample 36: using best label niche with score 0.718
2025-07-27 17:10:37 | reddit_moderation | INFO | Sample 42: using best label entertainment with score 0.736
2025-07-27 17:10:37 | reddit_moderation | INFO | Sample 43: using best label niche with score 0.726
2025-07-27 17:10:37 | reddit_moderation | INFO | Sample 54: using best label regional with score 0.780
2025-07-27 17:10:37 | reddit_moderation | INFO | Sample 59: using best label adult with score 0.735
2025-07-27 17:10:37 | reddit_moderation | INFO | Sample 63: using best label technology with score 0.722
2025-07-27 17:10:37 | reddit_moderation | INFO | Sample 76: using best label niche with score 0.768
2025-07-27 17:10:37 | reddit_moderation | INFO | Sample 95: using best label cr

Attach predictions:   0%|          | 0/10185 [00:00<?, ? examples/s]

# train test split

In [25]:
dataset = dataset.cast_column("violation", ClassLabel(names=["0", "1"]))

Casting the dataset:   0%|          | 0/10185 [00:00<?, ? examples/s]

In [26]:
splits = dataset.train_test_split(
    test_size=0.25, shuffle=True, seed=42, stratify_by_column="violation"
)
train = splits["train"]
test = splits["test"]

In [27]:
import torch
import gc

# Delete all objects
del subreddit_classifier
del rule_vectorizer  
del rule_vecs

# Force garbage collection
gc.collect()

# Clear GPU memory cache (if using CUDA)
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    
# Optional: Print memory usage to verify cleanup
if torch.cuda.is_available():
    print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
    print(f"GPU memory cached: {torch.cuda.memory_reserved()/1024**2:.2f} MB")


GPU memory allocated: 8.12 MB
GPU memory cached: 20.00 MB


# finetune text classification pipeline

In [28]:
# Training-specific logger
training_logger = create_logger(
    name="finetuning",
    log_level="DEBUG",
    log_file="training_session.log"
)

2025-07-27 17:10:39 | finetuning | INFO | Logger 'finetuning' created successfully
2025-07-27 17:10:39 | finetuning | INFO | Log level: DEBUG
2025-07-27 17:10:39 | finetuning | INFO | Log file: logs/training_session.log


In [29]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH["classifier"])

In [30]:
def _make_prompt(row):
    prompt = f"""Rule: {row['rule']}
Subreddit: {row['subreddit']} ({row['predicted_subreddit_feature']})
Content restrictions: {row['predicted_rule_feature']}

Comment: "{row['comment']}"

Question: Does this comment violate the rule?
Answer:"""
    return {"prompt": prompt}




In [31]:
def preprocess_(batch):
    return tokenizer(batch["prompt"], padding="max_length", truncation=True)




In [32]:
train = train.map(_make_prompt)
test = test.map(_make_prompt)

Map:   0%|          | 0/7638 [00:00<?, ? examples/s]

Map:   0%|          | 0/2547 [00:00<?, ? examples/s]

In [33]:
tokenized_train = train.map(preprocess_, batched=True, batch_size=128)
tokenized_test = test.map(preprocess_, batched=True, batch_size=128)

Map:   0%|          | 0/7638 [00:00<?, ? examples/s]

Map:   0%|          | 0/2547 [00:00<?, ? examples/s]

In [34]:
# Log your dataset info
training_logger.log_dataset_info(tokenized_train, "Training Dataset")
training_logger.log_dataset_info(tokenized_test, "Validation Dataset")

2025-07-27 17:10:45 | finetuning | INFO | Training Dataset Info:
2025-07-27 17:10:45 | finetuning | INFO |   - Size: 7,638 samples
2025-07-27 17:10:45 | finetuning | INFO |   - Columns: ['comment', 'rule', 'subreddit', 'violation', 'predicted_rule_feature', 'predicted_subreddit_feature', 'prompt', 'input_ids', 'attention_mask']
2025-07-27 17:10:45 | finetuning | INFO | Validation Dataset Info:
2025-07-27 17:10:45 | finetuning | INFO |   - Size: 2,547 samples
2025-07-27 17:10:45 | finetuning | INFO |   - Columns: ['comment', 'rule', 'subreddit', 'violation', 'predicted_rule_feature', 'predicted_subreddit_feature', 'prompt', 'input_ids', 'attention_mask']


In [35]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [36]:
columns = ["input_ids", "attention_mask", "violation"]
tokenized_train.set_format(type="torch", columns=columns)
tokenized_test.set_format(type="torch", columns=columns)

tokenized_train = tokenized_train.rename_column("violation", "labels")
tokenized_test = tokenized_test.rename_column("violation", "labels")

In [37]:
def compute_column_auc(eval_pred, logger=logger):
    print(f"{type(eval_pred) = }")
    logits, labels = eval_pred
    logger.info(f"{type(logits) = } | {type(labels) = } | {logits.shape = } | {labels.shape = }")
    probs = softmax(logits, axis=1)[:, 1]
    auc = roc_auc_score(y_true=labels, y_score=probs)
    return {"auc": auc}

In [38]:
id2label = {0: "not a violation", 1: "violation", 2:"NA"}
label2id = {"not a violation": 0, "violation": 1, "NA":2}
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH["classifier"], num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at /kaggle/input/facebookai-roberta-large-mnli/transformers/default/1 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
# Log model information
training_logger.log_model_info(model, "Classifier")

2025-07-27 17:10:46 | finetuning | INFO | Classifier Info:
2025-07-27 17:10:46 | finetuning | INFO |   - Model type: roberta
2025-07-27 17:10:46 | finetuning | INFO |   - Hidden size: 1024
2025-07-27 17:10:46 | finetuning | INFO |   - Number of labels: 3
2025-07-27 17:10:46 | finetuning | INFO |   - Total parameters: 355,362,819
2025-07-27 17:10:46 | finetuning | INFO |   - Trainable parameters: 355,362,819


In [40]:
# os.makedirs("finetuned-model", exist_ok=True)
# training_args = TrainingArguments(
#     output_dir="finetuned-model",
#     learning_rate=3e-5,  # Changed from 2e-3
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     gradient_accumulation_steps=4,
#     num_train_epochs=2,
#     weight_decay=0.01,
#     eval_strategy="steps",
#     eval_steps=50,
#     save_strategy="steps",
#     save_steps=200,
#     logging_steps=50,
#     load_best_model_at_end=True,
#     push_to_hub=False,
#     # Add these for better convergence
#     lr_scheduler_type="linear",
#     warmup_ratio=0.1,
#     report_to="none"
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_test,
#     processing_class=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_column_auc,
# )

In [41]:
# trainer.train()

In [42]:
# results = trainer.evaluate()
# print("Mean_column_AUC:", results["eval_auc"])

# now to train on entire dataset

In [43]:
# Combine your train and validation datasets
full_dataset = concatenate_datasets([tokenized_train, tokenized_test])

# Update your trainer configuration
os.makedirs("finetuned-model-full", exist_ok=True)
training_args = TrainingArguments(
    output_dir="finetuned-model-full",  # Different output dir
    # Learning rate - much lower for fine-tuning pre-trained model
    learning_rate=1e-5,                      # Conservative for large model
    # Batch configuration
    per_device_train_batch_size=4,           # Smaller due to model size
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,           # 4 * 8 = 32 effective batch size
    # Training duration
    num_train_epochs=3,                      # Start conservatively
    # Optimization
    weight_decay=0.01,                       # From original training
    adam_beta1=0.9,                         # From original training
    adam_beta2=0.98,                        # From original training (not default 0.999)
    # Learning rate scheduling
    lr_scheduler_type="linear",
    warmup_ratio=0.06,                      # ~6% warmup (similar to original)
    # Evaluation and logging
    # eval_strategy="steps",
    # eval_steps=100,                         # More frequent due to longer training
    save_strategy="steps", 
    save_steps=200,
    logging_steps=5,
    # Memory and performance
    fp16=True,                              # Essential for large model
    dataloader_pin_memory=False,            # Save memory
    load_best_model_at_end=False,
    # Monitoring
    report_to=None,
    logging_strategy="steps",
    logging_first_step=True,
)

# Log training configuration
training_logger.log_training_args(training_args)

class TrainingMetricsCallback(TrainerCallback):
    def __init__(self, train_dataset, tokenizer, model, compute_every_n_steps=50):
        self.train_dataset = train_dataset
        self.tokenizer = tokenizer
        self.model = model
        self.compute_every_n_steps = compute_every_n_steps

    def on_log(self, args, state, control, model=None, logs=None, **kwargs):
        """Compute training AUC every N steps"""
        if state.global_step % self.compute_every_n_steps == 0:
            # Sample a subset of training data for efficiency (optional)
            sample_size = min(1000, len(self.train_dataset))  # Use up to 1000 samples
            indices = np.random.choice(
                len(self.train_dataset), sample_size, replace=False
            )
            sample_dataset = self.train_dataset.select(indices)

            # Get predictions on training sample
            model.eval()
            all_logits = []
            all_labels = []

            # Process in batches to avoid memory issues
            batch_size = 32
            for i in range(0, len(sample_dataset), batch_size):
                batch = sample_dataset[i : i + batch_size]

                # Prepare inputs
                inputs = {
                    "input_ids": batch["input_ids"].clone().detach().to(model.device),
                    "attention_mask": batch["attention_mask"].clone().detach().to(
                        model.device
                    ),
                }

                with torch.no_grad():
                    outputs = model(**inputs)
                    logits = outputs.logits.cpu().numpy()
                    all_logits.append(logits)
                    all_labels.extend(batch["labels"])

            # Combine all predictions
            all_logits = np.vstack(all_logits)
            all_labels = np.array(all_labels)

            # Compute AUC using your existing function
            # eval_pred = type(
            #     "EvalPred", (), {"predictions": all_logits, "label_ids": all_labels}
            # )()
            metrics = compute_column_auc((all_logits, all_labels))

            # Log the training AUC
            if "auc" in metrics:
                eval_metrics = {
                    "step":state.global_step, 
                    "auc": metrics['auc'], 
                    "loss": logs.get('loss', 0), 
                    "LR": logs.get('learning_rate', 0)
                }
                training_logger.log_metrics(eval_metrics, "Validation")

            model.train()  # Switch back to training mode


# Updated trainer configuration
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_dataset,
    eval_dataset=None,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=None,
    callbacks=[
        TrainingMetricsCallback(
            full_dataset, tokenizer, model, compute_every_n_steps=5
        )
    ],
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


2025-07-27 17:10:46 | finetuning | INFO | Training Configuration:
2025-07-27 17:10:46 | finetuning | INFO |   - Learning rate: 1e-05
2025-07-27 17:10:46 | finetuning | INFO |   - Batch size: 4
2025-07-27 17:10:46 | finetuning | INFO |   - Gradient accumulation: 8
2025-07-27 17:10:46 | finetuning | INFO |   - Epochs: 3
2025-07-27 17:10:46 | finetuning | INFO |   - Weight decay: 0.01
2025-07-27 17:10:46 | finetuning | INFO |   - LR scheduler: SchedulerType.LINEAR
2025-07-27 17:10:46 | finetuning | INFO |   - Warmup ratio: 0.06


In [44]:
trainer.train()

Step,Training Loss
1,1.114300
5,1.119100
10,0.821000
15,0.738500
20,0.705800
25,0.646200
30,0.655400
35,0.613000
40,0.568300
45,0.544800


type(eval_pred) = <class 'tuple'>
2025-07-27 17:13:20 | reddit_moderation | INFO | type(logits) = <class 'numpy.ndarray'> | type(labels) = <class 'numpy.ndarray'> | logits.shape = (1000, 3) | labels.shape = (1000,)
2025-07-27 17:13:20 | finetuning | INFO | Validation Metrics:
2025-07-27 17:13:20 | finetuning | INFO |   - step: 5
2025-07-27 17:13:20 | finetuning | INFO |   - auc: 0.3974
2025-07-27 17:13:20 | finetuning | INFO |   - loss: 1.1191
2025-07-27 17:13:20 | finetuning | INFO |   - LR: 0.0000
type(eval_pred) = <class 'tuple'>
2025-07-27 17:15:44 | reddit_moderation | INFO | type(logits) = <class 'numpy.ndarray'> | type(labels) = <class 'numpy.ndarray'> | logits.shape = (1000, 3) | labels.shape = (1000,)
2025-07-27 17:15:44 | finetuning | INFO | Validation Metrics:
2025-07-27 17:15:44 | finetuning | INFO |   - step: 10
2025-07-27 17:15:44 | finetuning | INFO |   - auc: 0.5470
2025-07-27 17:15:44 | finetuning | INFO |   - loss: 0.8210
2025-07-27 17:15:44 | finetuning | INFO |   - 

TrainOutput(global_step=480, training_loss=0.1738748471563061, metrics={'train_runtime': 13825.9931, 'train_samples_per_second': 2.21, 'train_steps_per_second': 0.035, 'total_flos': 2.847525901807104e+16, 'train_loss': 0.1738748471563061, 'epoch': 3.0})

# make predictions

In [45]:
logger.info("Starting test data processing and prediction pipeline")

# Load test data
logger.info("Loading test dataset")
test = pd.read_csv(os.path.join(INPUT_PATH, "jigsaw-agile-community-rules", "test.csv"))
logger.info(f"Test data loaded: {len(test)} rows, {len(test.columns)} columns")

# Sanitize comments
logger.info("Starting comment sanitization for all text columns")
for c in [
    "body",
    "rule",
    "subreddit",
    "positive_example_1",
    "positive_example_2",
    "negative_example_1",
    "negative_example_2",
]:
    logger.debug(f"Sanitizing column: {c}")
    test[c] = test[c].apply(sanitize_comment)
logger.info("Comment sanitization completed for all columns")

# Convert to HuggingFace Dataset
logger.debug("Converting pandas DataFrame to HuggingFace Dataset")
test = Dataset.from_pandas(test)
logger.debug(f"Dataset created with columns: {test.column_names}")

# Rename body column
logger.debug("Renaming 'body' column to 'comment'")
test = test.rename_column("body", "comment")

# Add classification predictions
logger.info("Starting rule and subreddit classification prediction")
logger.debug(f"Using existing rule_lookup with {len(rule_lookup)} entries")
logger.debug(f"Using existing subreddit_lookup with {len(subreddit_lookup)} entries")

test, rule_lookup, subreddit_lookup = add_classification_preds_rule_subreddit(
    test, rule_lookup=rule_lookup, subreddit_lookup=subreddit_lookup
)
logger.info("Classification predictions added successfully")
logger.debug(f"Test dataset now has columns: {test.column_names}")

# Create prompts
logger.info("Creating prompts for test data")
test = test.map(_make_prompt)
logger.debug(f"Prompts created, sample prompt length: {len(test[0]['text']) if 'text' in test.column_names else 'N/A'}")

# Tokenize test data
logger.info("Starting tokenization of test data")
tokenized_test = test.map(preprocess_, batched=True, batch_size=128)
logger.info(f"Tokenization completed: {len(tokenized_test)} samples")
logger.debug(f"Tokenized dataset columns: {tokenized_test.column_names}")

# Set format for PyTorch
logger.debug("Setting PyTorch tensor format")
columns = ["input_ids", "attention_mask"]
tokenized_test.set_format(type="torch", columns=columns)
logger.debug(f"Tensor format set for columns: {columns}")

# Generate predictions
logger.info("Starting model predictions on test data")
logger.debug(f"Using trainer with model: {type(trainer.model).__name__}")
predictions = trainer.predict(tokenized_test).predictions
logger.info(f"Predictions generated: shape {predictions.shape}")
logger.debug(f"Prediction range: [{predictions.min():.4f}, {predictions.max():.4f}]")

# Convert to probabilities
logger.debug("Converting logits to probabilities using softmax")
probs = softmax(predictions, axis=1)[:, 1]  # Get violation probabilities
logger.info(f"Probabilities computed: {len(probs)} samples")
logger.debug(f"Probability range: [{probs.min():.4f}, {probs.max():.4f}]")
logger.debug(f"Mean probability: {probs.mean():.4f}")

# Create submission file
logger.info("Creating submission DataFrame")
sub = pd.DataFrame({"row_id": test["row_id"], "rule_violation": probs})
logger.info(f"Submission DataFrame created: {len(sub)} rows")
logger.debug(f"Row ID range: {sub['row_id'].min()} to {sub['row_id'].max()}")

# Save submission
logger.info("Saving submission to CSV file")
sub.to_csv("submission.csv", index=False)
logger.info("Submission file 'submission.csv' saved successfully")

# Display sample results
logger.info("Displaying sample submission results:")
sample_results = sub.head()
for idx, row in sample_results.iterrows():
    logger.info(f"  Row ID {row['row_id']}: Violation probability = {row['rule_violation']:.4f}")

logger.info("Test data processing and prediction pipeline completed successfully")

# Optional: Log summary statistics
logger.info("=== PREDICTION SUMMARY ===")
logger.info(f"Total test samples processed: {len(sub)}")
logger.info(f"High confidence violations (>0.8): {(probs > 0.8).sum()}")
logger.info(f"Medium confidence violations (0.5-0.8): {((probs > 0.5) & (probs <= 0.8)).sum()}")
logger.info(f"Low violations (<0.5): {(probs <= 0.5).sum()}")
logger.info("=== END SUMMARY ===")

sub.head()

2025-07-27 21:02:48 | reddit_moderation | INFO | Starting test data processing and prediction pipeline
2025-07-27 21:02:48 | reddit_moderation | INFO | Loading test dataset
2025-07-27 21:02:48 | reddit_moderation | INFO | Test data loaded: 10 rows, 8 columns
2025-07-27 21:02:48 | reddit_moderation | INFO | Starting comment sanitization for all text columns
2025-07-27 21:02:48 | reddit_moderation | INFO | Comment sanitization completed for all columns
2025-07-27 21:02:48 | reddit_moderation | INFO | Starting rule and subreddit classification prediction


Attach predictions:   0%|          | 0/10 [00:00<?, ? examples/s]

2025-07-27 21:02:48 | reddit_moderation | INFO | Classification predictions added successfully
2025-07-27 21:02:48 | reddit_moderation | INFO | Creating prompts for test data


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

2025-07-27 21:02:48 | reddit_moderation | INFO | Starting tokenization of test data


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

2025-07-27 21:02:48 | reddit_moderation | INFO | Tokenization completed: 10 samples
2025-07-27 21:02:48 | reddit_moderation | INFO | Starting model predictions on test data


2025-07-27 21:02:49 | reddit_moderation | INFO | Predictions generated: shape (10, 3)
2025-07-27 21:02:49 | reddit_moderation | INFO | Probabilities computed: 10 samples
2025-07-27 21:02:49 | reddit_moderation | INFO | Creating submission DataFrame
2025-07-27 21:02:49 | reddit_moderation | INFO | Submission DataFrame created: 10 rows
2025-07-27 21:02:49 | reddit_moderation | INFO | Saving submission to CSV file
2025-07-27 21:02:49 | reddit_moderation | INFO | Submission file 'submission.csv' saved successfully
2025-07-27 21:02:49 | reddit_moderation | INFO | Displaying sample submission results:
2025-07-27 21:02:49 | reddit_moderation | INFO |   Row ID 2029.0: Violation probability = 0.0042
2025-07-27 21:02:49 | reddit_moderation | INFO |   Row ID 2030.0: Violation probability = 0.3374
2025-07-27 21:02:49 | reddit_moderation | INFO |   Row ID 2031.0: Violation probability = 0.9988
2025-07-27 21:02:49 | reddit_moderation | INFO |   Row ID 2032.0: Violation probability = 0.9991
2025-07-2

,row_id,rule_violation
0,2029,0.004227
1,2030,0.337413
2,2031,0.998847
3,2032,0.999130
4,2033,0.999126
